<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with a custom metrics provider

This notebook demonstrates how to configure the custom monitor and custom metrics deployment by using IBM Watson OpenScale. It requires service credentials for the following services:
  * Watson OpenScale
  * Watson Machine Learning

## 1. Set up your environment. <a name="setup"></a>

Before you use the sample code in this notebook, you must perform the following setup tasks:

### Install the  `ibm-watson-machine-learning` and `ibm-watson-openscale` packages.

In [ ]:
!pip install --upgrade ibm-watson-machine-learning   | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade wget --no-cache | tail -n 1

### Action: restart the kernel!

### Credentials for IBM Cloud Pak for Data
To authenticate, in the following code boxes, replace the sample data with your own credentials. Get the information from your system administrator or through the Cloud Pak for Data dashboard.


### Obtaining your Watson OpenScale credentials

You can retrieve the URL by running the following command: `oc get route -n namespace1 --no-headers | awk '{print $2}'` Replace the `namespace1` variable with your namespace.

You should have been assigned a username and password when you were added to the Cloud Pak for Data system. You might need to ask either your database administrator or your system administrator for some of the information.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
username = 'admin'
api_key = 'xxxxxxxxx'
url = 'https://cpd-namespace1.apps.xxxxxxxxxxxx.cp.fyre.ibm.com'
password = 'password'

### Python function details

In [ ]:
PYTHON_FUNCTION_NAME = 'Custom Metrics Provider Function'
DEPLOYMENT_NAME = 'Custom Metrics Provider Deployment'

### OpenScale Custom Metrics Provider name

In [ ]:
CUSTOM_METRICS_PROVIDER_NAME = "Custom Metrics Provider"

### OpenSale Custom Monitor name

In [ ]:
###################################################################
# UPDATE your custom monitor name in the following field and then run this cell.
####################################################################
CUSTOM_MONITOR_NAME = 'Sample Model Performance'

In [ ]:
############################################################################################
# Paste your Watson OpenScale credentials into the following section and then run this cell.
############################################################################################
WOS_CREDENTIALS = {
    "username": username,
    "apikey": api_key,
    "url": url
}

In [ ]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='4.0'
WML_CREDENTIALS

# 1. Create the custom metrics provider - Python function.

The Python function receives the required variables, such as the `datamart_id`, `monitor_id`, `monitor_instance_id`, `monitor_run_id`, `monitor_instance_parameters` and `subscription_id` from the Watson OpenScale service when it is invoked by the custom monitor. 

In the Python function, add your own logic to compute the custom metrics in the `get_metrics` method, publish the metrics to the Watson Openscale service and update the status of the run to the `finished` state in the custom monitor instance run.

Update the `WOS_CREDENTIALS` in the Python function. 

In [ ]:
#wml_python_function
parms = {
        "url": WOS_CREDENTIALS["url"],
        "username": WOS_CREDENTIALS["username"],
        "apikey": WOS_CREDENTIALS["apikey"]
    }
def custom_metrics_provider(parms = parms):
    
    import json
    import requests
    import base64
    from requests.auth import HTTPBasicAuth
    import time
    import uuid
    import datetime
    
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Accept"] = "application/json"
    
    
    # Get the access token
    def get_access_token():
        url = '{}/icp4d-api/v1/authorize'.format(parms['url'])
        payload = {
            'username': parms['username'],
            'api_key': parms['apikey']
        }
        response = requests.post(url, headers=headers, json=payload, verify=False)
        json_data = response.json()
        access_token = json_data['token']
        return access_token
    
    def get_feedback_dataset_id(access_token, data_mart_id, subscription_id):
        headers["Authorization"] = "Bearer {}".format(access_token)
        DATASETS_URL =  parms["url"] + "/openscale/{0}/v2/data_sets?target.target_id={1}&target.target_type=subscription&type=feedback".format(data_mart_id, subscription_id)
        response = requests.get(DATASETS_URL, headers=headers, verify=False)
        json_data = response.json()
        feedback_dataset_id = None
        if "data_sets" in json_data and len(json_data["data_sets"]) > 0:
            feedback_dataset_id = json_data["data_sets"][0]["metadata"]["id"]
        
        return feedback_dataset_id
    
    def get_feedback_data(access_token, data_mart_id, feedback_dataset_id):
        json_data = None
        if feedback_dataset_id is not None:
            headers["Authorization"] = "Bearer {}".format(access_token)
            DATASETS_STORE_RECORDS_URL = parms["url"] + "/openscale/{0}/v2/data_sets/{1}/records?limit={2}&format=list".format(data_mart_id, feedback_dataset_id, 100)
            response = requests.get(DATASETS_STORE_RECORDS_URL, headers=headers, verify=False)
            json_data = response.json()
            return json_data
    
    #Update the run status to Finished in the Monitor Run
    def update_monitor_run_status(base_url, access_token, custom_monitor_instance_id, run_id, status, error_msg = None):
        monitor_run_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/runs/'+run_id
        completed_timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        patch_payload  = []
        base_path = "/status"
        
        patch_payload.append(get_patch_request_field(base_path, "state", status))
        patch_payload.append(get_patch_request_field(base_path, "completed_at", completed_timestamp))
        if error_msg != None:
            error_json = get_error_json(error_msg)
            patch_payload.append(get_patch_request_field(base_path, "failure", error_json))
        
        headers["Authorization"] = "Bearer {}".format(access_token)
        response = requests.patch(monitor_run_url, headers=headers, json = patch_payload, verify=False)
        monitor_run_response = response.json()
        return response.status_code, monitor_run_response
    
    
    def get_error_json(error_message):
        trace = str(uuid.uuid4())
        error_json = {
            'trace': trace,
            'errors': [{
                'code': "custom_metrics_error_code",
                'message': str(error_message)
            }]
        }
        return error_json
    
    def get_patch_request_field(base_path, field_name, field_value, op_name="replace"):
        field_json = {
            "op": op_name,
            "path": "{0}/{1}".format(base_path, field_name),
            "value": field_value
        }
        return field_json
    
    #Add your code to compute the custom metrics. 
    def get_metrics(access_token, data_mart_id, subscription_id):
        #Add the logic here to compute the metrics. Use the below metric names while creating the custom monitor definition
        feedback_dataset_id = get_feedback_dataset_id(access_token, data_mart_id, subscription_id)
        json_data = get_feedback_data(access_token, data_mart_id, feedback_dataset_id)
        gender_less40_fav_prediction_ratio = 0
        if json_data is not None:
            fields = json_data['records'][0]['fields']
            values = json_data['records'][0]['values']
            import pandas as pd
            feedback_data = pd.DataFrame(values, columns = fields)
            female_less40_fav_prediction = len(feedback_data.query('Sex == \'female\' & Age <= 40 & Risk == \'No Risk\''))
            male_less40_fav_prediction = len(feedback_data.query('Sex == \'male\' & Age <= 40 & Risk == \'No Risk\''))
            gender_less40_fav_prediction_ratio = female_less40_fav_prediction / male_less40_fav_prediction
        
        #Remove the tag("region": "us-south") in below metrics while publishing the metric values to Openscale Datamart 
        #if the custom monitor definition is not created with tags
        metrics = {"specificity": 1.2, "sensitivity": 0.85, "gender_less40_fav_prediction_ratio": gender_less40_fav_prediction_ratio, "region": "us-south"}
        #metrics = {"specificity": 1.2, "sensitivity": 0.85, "gender_less40_fav_prediction_ratio": gender_less40_fav_prediction_ratio}
        
        return metrics
        
        
    # Publishes the Custom Metrics to OpenScale
    def publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitoring_run_id, timestamp):
        # Generate an monitoring run id, where the publishing happens against this run id
        custom_metrics = get_metrics(access_token, data_mart_id, subscription_id)
        measurements_payload = [
                  {
                    "timestamp": timestamp,
                    "run_id": custom_monitoring_run_id,
                    "metrics": [custom_metrics]
                  }
                ]
        headers["Authorization"] = "Bearer {}".format(access_token)
        measurements_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/measurements'
        response = requests.post(measurements_url, headers=headers, json = measurements_payload, verify=False)
        published_measurement = response.json()
     
        return response.status_code, published_measurement
        
    
    def publish( input_data ):
        
        timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        
        payload = input_data.get("input_data")[0].get("values")
        data_mart_id = payload['data_mart_id']
        subscription_id = payload['subscription_id']
        custom_monitor_id = payload['custom_monitor_id']
        custom_monitor_instance_id = payload['custom_monitor_instance_id']
        custom_monitor_instance_params  = payload['custom_monitor_instance_params']
        custom_monitor_run_id = payload['custom_monitor_run_id']

        base_url = parms['url'] + '/openscale' + '/' + data_mart_id
        access_token = get_access_token()
        
        published_measurements = []
        error_msgs = []
        run_status = "finished"
        
        try:
            last_run_time = custom_monitor_instance_params.get("last_run_time")
            max_records = custom_monitor_instance_params.get("max_records")
            min_records = custom_monitor_instance_params.get("min_records")
            error_msg = None
            
            status_code, published_measurement = publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitor_run_id, timestamp)
            if int(status_code) in [200, 201, 202]:
                published_measurements.append(published_measurement)
            else:
                run_status = "error"
                error_msg = published_measurement
                error_msgs.append(error_msg)
                    
            status_code, response = update_monitor_run_status(base_url, access_token, custom_monitor_instance_id, custom_monitor_run_id, run_status, error_msg)
            
            if not int(status_code) in [200, 201, 202]:
                error_msgs.append(response)
                
        except Exception as ex:
            error_msgs.append(str(ex))
        if len(error_msgs) == 0:
            response_payload = {
                "predictions" : [{ 
                    "values" : published_measurements
                }]

            }
        else:
            response_payload = {
                "predictions":[],
                "errors": error_msgs
            }
        
        return response_payload
        
    return publish
    

## Register the custom metrics provider and create a deployment. <a name="deployment"></a>

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
wml_client.spaces.list(limit=10)

In [ ]:
space_id = 'eff8e06f-2572-445a-8d77-ec0dc8e18c8b'
wml_client.set.default_space(space_id)

### Remove existing function and deployment.

In [ ]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)

wml_client.repository.list_functions()

### Create the function meta properties.


In [ ]:
software_spec_id =  wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')
print(software_spec_id)
function_meta_props = {
     wml_client.repository.FunctionMetaNames.NAME: PYTHON_FUNCTION_NAME,
     wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: software_spec_id
     }

### Store the Python function.

In [ ]:
function_artifact = wml_client.repository.store_function(meta_props=function_meta_props, function=custom_metrics_provider)
function_uid = wml_client.repository.get_function_id(function_artifact)
print("Function UID = " + function_uid)

In [ ]:
function_details = wml_client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

### Deploy the Python function.


In [ ]:
hardware_spec_id = wml_client.hardware_specifications.get_id_by_name('M')
hardware_spec_id

### Create deployment metadata for the Python function.

In [ ]:
deploy_meta = {
 wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
 wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
 wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id}
}

### Create a deployment.

In [ ]:
deployment_details = wml_client.deployments.create(function_uid, meta_props=deploy_meta)

### Get the python function scoring URL.

In [ ]:
created_at = deployment_details['metadata']['created_at']
find_string_pos = created_at.find("T")
if find_string_pos != -1:
    current_date = created_at[0:find_string_pos]
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
scoring_url = scoring_url + "?version="+current_date
print(scoring_url)


# 2. Generate training data schema for configuring OpenScale

### Describe the training data

In [ ]:
feature_columns=["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
cat_features=["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
class_label = "Risk"

### Get the training data

In [ ]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv

In [ ]:
import pandas as pd
data_df=pd.read_csv ("german_credit_data_biased_training.csv")
data_df.head()

### Generate training data schema

In [ ]:
from ibm_watson_openscale.utils.training_stats import TrainingStats

In [ ]:
input_parameters = {
    "label_column": class_label,
    "feature_columns": feature_columns,
    "categorical_columns": cat_features,
    "fairness_inputs": None,  
    "problem_type" : "binary"
}

In [ ]:
training_stats = TrainingStats(data_df,input_parameters, explain=False, fairness=False, drop_na=True)

In [ ]:
config_json = training_stats.get_training_statistics()

In [ ]:
config_json["notebook_version"] = 5.0

In [ ]:
config_json

# 3. Configure OpenScale 

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

In [ ]:
WOS_GUID = '00000000-0000-0000-0000-1661432974606959'

## Get a instance of the OpenScale SDK client

In [ ]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        apikey=WOS_CREDENTIALS['apikey'],
        disable_ssl_verification=True,
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator, service_instance_id = WOS_GUID)
wos_client.version

## Set up datamart

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
data_mart_id=data_marts[0].metadata.id
print('Using existing datamart {}'.format(data_mart_id))

In [ ]:
data_mart_details = wos_client.data_marts.list().result.data_marts[0]
data_mart_details.to_dict()

In [ ]:
wos_client.service_providers.show()

## Remove existing service provider

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one.

In [ ]:
SERVICE_PROVIDER_NAME = "RC - OpenScale Headless Service Provider"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook to showcase Headless Subscription functionality."

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

Note: Here the service provider is created with empty credentials, meaning no endpoint. Just to demonstrate the use case were we don't need an actual end point serving requests.

In [ ]:
MLCredentials = {}
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        operational_space_id = "production",
        credentials=MLCredentials,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
print(wos_client.service_providers.get(service_provider_id).result)

## Subscriptions

Remove existing credit risk subscriptions

This code removes previous subscriptions to the model to refresh the monitors with the new model and new data.

In [ ]:
wos_client.subscriptions.show()

## Remove the existing subscription

In [ ]:
SUBSCRIPTION_NAME = "RC - GCR Headless Subscription"

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    if subscription.entity.asset.name == '[asset] ' + SUBSCRIPTION_NAME:
        sub_model_id = subscription.metadata.id
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

In [ ]:
print("Data Mart ID: " + data_mart_id)
print("Service Provide ID: " + service_provider_id)
import uuid
asset_id = str(uuid.uuid4())
asset_name = '[asset] ' + SUBSCRIPTION_NAME
url = None

asset_deployment_id = str(uuid.uuid4())
asset_deployment_name = asset_name

In [ ]:
prediction_column = "prediction"
probability_columns = ['probability']
predicted_target_column = "prediction"
subscription_details = wos_client.subscriptions.add(data_mart_id,
    service_provider_id,
    asset=Asset(
        asset_id=asset_id,
        name=asset_name,
        url=url,
        asset_type=AssetTypes.MODEL,
        input_data_type=InputDataType.STRUCTURED,
        problem_type=ProblemType.BINARY_CLASSIFICATION
    ),
    deployment=AssetDeploymentRequest(
        deployment_id=asset_deployment_id,
        name=asset_deployment_name,
        deployment_type= DeploymentTypes.ONLINE
    ),
    training_data_stats=config_json,
    prediction_field = prediction_column,
    predicted_target_field = predicted_target_column,
    probability_fields = probability_columns,
    background_mode = False,
    deployment_name = asset_name
    ).result

subscription_id = subscription_details.metadata.id
print("Subscription id {}".format(subscription_id))

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

In [ ]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

## Push a payload record to setup the required schemas in the subscription

Note : No scoring is done against the model. The PayloadRecord is constructed with the request and response from the model/deployment.

## Scoring Request Payload

In [ ]:
scoring_request =   {
        "fields": [
            "CheckingStatus",
            "LoanDuration",
            "CreditHistory",
            "LoanPurpose",
            "LoanAmount",
            "ExistingSavings",
            "EmploymentDuration",
            "InstallmentPercent",
            "Sex",
            "OthersOnLoan",
            "CurrentResidenceDuration",
            "OwnsProperty",
            "Age",
            "InstallmentPlans",
            "Housing",
            "ExistingCreditsCount",
            "Job",
            "Dependents",
            "Telephone",
            "ForeignWorker",
            "Risk"
        ],
        "values": [
            [
                "no_checking",
                28,
                "outstanding_credit",
                "appliances",
                5990,
                "500_to_1000",
                "greater_7",
                5,
                "male",
                "co-applicant",
                3,
                "car_other",
                55,
                "none",
                "free",
                2,
                "skilled",
                2,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "greater_200",
                22,
                "all_credits_paid_back",
                "car_used",
                3376,
                "less_100",
                "less_1",
                3,
                "female",
                "none",
                2,
                "car_other",
                32,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ]
        ],
        "meta": {
            "fields": [
                "referrer_gender"
            ],
            "values": [
                [
                    "male"
                ],
                [
                    "female"
                ]
            ]
        }
    }

## Scoring Response Payload

In [ ]:
scoring_response = {
    "predictions": [
        {
            "fields": [
                "prediction",
                "probability"
            ],
            "values": [
                [
                    "Risk",
                    [
                        0.104642951112211,
                        0.895357048887789
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.892112895920181,
                        0.10788710407981907
                    ]
                ]
            ]
        }
    ]
}

In [ ]:
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

records_list=[]
for x in range(10):
    pl_record = PayloadRecord(request=scoring_request, response=scoring_response)
    records_list.append(pl_record)

wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=records_list)

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

## Fetch the subscription details to confirm schemas are setup

In [ ]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

## Enable feedback logging

In [ ]:
wos_client.subscriptions.create_feedback_table(subscription_id)

## Get feedback logging dataset ID

In [ ]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [ ]:
feedback_dataset_id

In [ ]:
feedback_payload = {
    "fields": [
        "CheckingStatus",
        "LoanDuration",
        "CreditHistory",
        "LoanPurpose",
        "LoanAmount",
        "ExistingSavings",
        "EmploymentDuration",
        "InstallmentPercent",
        "Sex",
        "OthersOnLoan",
        "CurrentResidenceDuration",
        "OwnsProperty",
        "Age",
        "InstallmentPlans",
        "Housing",
        "ExistingCreditsCount",
        "Job",
        "Dependents",
        "Telephone",
        "ForeignWorker",
        "Risk",
        "_original_probability",
        "_original_prediction",
        "_debiased_probability",
        "_debiased_prediction"
    ],
    "values": [
        [
            "less_0",
            18,
            "credits_paid_to_date",
            "car_new",
            462,
            "less_100",
            "1_to_4",
            2,
            "female",
            "none",
            2,
            "savings_insurance",
            37,
            "stores",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            15,
            "prior_payments_delayed",
            "furniture",
            250,
            "less_100",
            "1_to_4",
            2,
            "male",
            "none",
            3,
            "real_estate",
            28,
            "none",
            "own",
            2,
            "skilled",
            1,
            "yes",
            "no",
            "No Risk",
            [
                0.7419002139563244,
                0.25809978604367556
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "0_to_200",
            28,
            "credits_paid_to_date",
            "retraining",
            3693,
            "less_100",
            "greater_7",
            3,
            "male",
            "none",
            2,
            "savings_insurance",
            32,
            "none",
            "own",
            1,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6935080115729353,
                0.3064919884270647
            ],
            "Risk",
            [
                0.8,
                0.2
            ],
            "Risk"
        ],
        [
            "no_checking",
            28,
            "prior_payments_delayed",
            "education",
            6235,
            "500_to_1000",
            "greater_7",
            3,
            "male",
            "none",
            3,
            "unknown",
            57,
            "none",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "Risk",
            [
                0.331110352092386,
                0.668889647907614
            ],
            "Risk",
            [
                0.9,
                0.1
            ],
            "Risk"
        ],
        [
            "no_checking",
            32,
            "outstanding_credit",
            "vacation",
            9604,
            "500_to_1000",
            "greater_7",
            6,
            "male",
            "co-applicant",
            5,
            "unknown",
            57,
            "none",
            "free",
            2,
            "skilled",
            2,
            "yes",
            "yes",
            "Risk",
            [
                0.11270206970758759,
                0.8872979302924124
            ],
            "Risk",
            [
                0.1,
                0.9
            ],
            "Risk"
        ],
        [
            "no_checking",
            9,
            "prior_payments_delayed",
            "car_new",
            1032,
            "100_to_500",
            "4_to_7",
            3,
            "male",
            "none",
            4,
            "savings_insurance",
            41,
            "none",
            "own",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6704819620865308,
                0.32951803791346923
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            16,
            "credits_paid_to_date",
            "vacation",
            3109,
            "less_100",
            "4_to_7",
            3,
            "female",
            "none",
            1,
            "car_other",
            36,
            "none",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6735810290914039,
                0.3264189709085961
            ],
            "Risk",
            [
                0.6,
                0.4
            ],
            "Risk"
        ],
        [
            "0_to_200",
            11,
            "credits_paid_to_date",
            "car_new",
            4553,
            "less_100",
            "less_1",
            3,
            "female",
            "none",
            3,
            "savings_insurance",
            22,
            "none",
            "own",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.637964656269084,
                0.362035343730916
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "no_checking",
            35,
            "outstanding_credit",
            "appliances",
            7138,
            "500_to_1000",
            "greater_7",
            5,
            "male",
            "co-applicant",
            4,
            "unknown",
            49,
            "none",
            "free",
            2,
            "skilled",
            2,
            "yes",
            "yes",
            "Risk",
            [
                0.11270206970758759,
                0.8872979302924124
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            5,
            "all_credits_paid_back",
            "car_new",
            1523,
            "less_100",
            "unemployed",
            2,
            "female",
            "none",
            2,
            "real_estate",
            19,
            "none",
            "rent",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.7304597628653227,
                0.26954023713467745
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ]
    ]
}

In [ ]:
fields = feedback_payload['fields']
values = feedback_payload['values']

In [ ]:
import pandas as pd
scored_feedback_data = pd.DataFrame(values, columns = fields)

In [ ]:
len(scored_feedback_data)

In [ ]:
scored_feedback_data = scored_feedback_data.loc[scored_feedback_data.index.repeat(2)]

In [ ]:
len(scored_feedback_data)

In [ ]:
fields = scored_feedback_data.columns.tolist()
values = scored_feedback_data[fields].values.tolist()

feedback_payload = {"fields": fields, "values": values}

### Store the feedback payload using the data sets API

In [ ]:
import urllib3, requests, json
def generate_access_token():
    headers={}
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth(username, password)
    
    ICP_TOKEN_URL= WOS_CREDENTIALS["url"] + "/v1/preauth/validateAuth"
    
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    icp_access_token = json_data['accessToken']
    return icp_access_token

In [ ]:
icp_access_token = generate_access_token()

In [ ]:
header = {
    'Content-Type': 'application/json', 
    'Authorization': 'Bearer ' + icp_access_token
}

In [ ]:
DATASETS_STORE_RECORDS_URL =   WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/data_sets/{1}/records".format(data_mart_id, feedback_dataset_id)
for x in range(10):
    response = requests.post(DATASETS_STORE_RECORDS_URL, json=feedback_payload, headers=header, verify=False)
    json_data = response.json()
    print(json_data)
    time.sleep(3)

### Wait for sometime, and make sure the records have reached to data sets related table.

In [ ]:
time.sleep(10)
DATASETS_STORE_RECORDS_URL =   WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/data_sets/{1}/records?limit={2}&include_total_count={3}".format(data_mart_id, feedback_dataset_id, 1, "true")
response = requests.get(DATASETS_STORE_RECORDS_URL, headers=header, verify=False)
json_data = response.json()
print(json_data['total_count'])

# Register OpenScale subscription with the Custom Metrics Provider

You can configure custom metrics for the subscriptions that have predefined monitors, such as fairness, quality, or drift or without predefined monitors.

## Configure Custom Metrics Provider with Watson OpenScale.

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMetricRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MetricThreshold
from ibm_watson_openscale.supporting_classes.enums import MetricThresholdTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorTagRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Target
from ibm_watson_openscale.supporting_classes.enums import TargetTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import IntegratedSystems
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorInstanceSchedule, ScheduleStartTime, MonitorRuntime

from datetime import datetime, timezone, timedelta
import uuid

## Create the integrated system for the custom metrics provider. <a name="custom"></a>


Update the custom metrics deployment URL, which is created during the Python function creation in the integrated system. Watson OpenScale invokes the deployment URL at runtime to compute the custom metrics. 

You must define the authentication type based on the communication with custom metrics deployment. Watson OpenScale supports 2 types of authentication: basic and bearer. If custom metrics deployment accepts the `basic` authentication type, then provide `auth_type=basic` otherwise use `auth_type=bearer`.

In [ ]:
CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
    "auth_type":"bearer",
    "token_info": {
        "url": "{}/icp4d-api/v1/authorize".format(WOS_CREDENTIALS['url']),
        "headers": {
            "Content-Type": "application/json",
            "Accept": "application/json"
        },
        "payload": {
            "username": WOS_CREDENTIALS['username'],
            "api_key": WOS_CREDENTIALS['apikey'],
        },
        "method": "post"
    }
}

### Remove existing integrated system 

In [ ]:
# Delete existing custom metrics provider integrated systems if present
integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems
for system in integrated_systems:
    if system.entity.type == 'custom_metrics_provider' and system.entity.name == CUSTOM_METRICS_PROVIDER_NAME:
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

In [ ]:
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=CUSTOM_METRICS_PROVIDER_NAME,
    description=CUSTOM_METRICS_PROVIDER_NAME,
    type="custom_metrics_provider",
    credentials= CUSTOM_METRICS_PROVIDER_CREDENTIALS,
    connection={
        "display_name": CUSTOM_METRICS_PROVIDER_NAME,
        "endpoint": scoring_url
    }
).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(custom_metrics_integrated_system)

## Set up the custom monitor definition and instance. <a name="instance"></a>


### Check for the existence of the custom monitor definition.


In [ ]:
def get_custom_monitor_definition():
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    for definition in monitor_definitions:
        if CUSTOM_MONITOR_NAME == definition.entity.name:
            return definition
    return None   

### Create the  custom monitor definition.

Update the custom metric names, threshold types (`LOWER_LIMIT`, `UPPER_LIMIT`) and default values as required. You can define the threshold type as lower limit, upper limit, or both.

In [ ]:
###################################################################
# Update your custom monitor metrics names in the following field. Use the same metric names for creating the 
# monitor definition and publishing the metrics to Openscale in your python function
####################################################################
CUSTOM_MONITOR_METRICS_NAMES = ['sensitivity','specificity', 'gender_less40_fav_prediction_ratio']
#Update the tag values if you want to fetch the metrics by tags
TAGS= ['region']
TAG_DESCRIPTION =['customer geographical region'] 

### Create  schedule for custom monitor

Creates a schedule for custom monitor with a defined repeat type and repeat interval. Supported repeat types are "minute", "hour", "day", "week", "month", "year". Defaults to once every hour if `repeat_interval` and  `repeat_type` values are not specified


In [ ]:
#Enable the schedule for your custom monitor. Update the value to false if you want to disable the schedule
ENABLE_SCHEDULE = True

#Update the repeat interval and repeat type: Default is 1 hour
repeat_interval = 1
repeat_type = "hour"

#Update the delay unit and interval to trigger the first schedule run after the custom monitor instance is created. Default is 30 minutes
delay_unit = "minute"
delay_time= 30

start_time= ScheduleStartTime(type = "relative", delay_unit= delay_unit, delay = delay_time)
schedule = MonitorInstanceSchedule(repeat_interval=repeat_interval,repeat_unit=repeat_type, start_time=start_time)    

In [ ]:
#Update the Threshold types and default values of the metrics

def custom_metric_definitions():
    
    metrics = [MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[0],
                                    thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
              MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[1],
                                 thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.6),MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=1)]),
              MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[2],
                                 thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.6),MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=1)])]
    
    #Comment the below tags code if there are no tags to be created
    tags = [MonitorTagRequest(name=TAGS[0], description=TAG_DESCRIPTION[0])]
    
    return metrics, tags

In [ ]:
def create_custom_monitor_definition():
    # check if the custom monitor definition already exists or not
    existing_definition = get_custom_monitor_definition()

    # if it does not exists, then create a new one.
    if existing_definition is None:
        metrics, tags = custom_metric_definitions()
        if ENABLE_SCHEDULE:
            monitor_runtime = MonitorRuntime(type="custom_metrics_provider")
            custom_monitor_details = wos_client.monitor_definitions.add(name=CUSTOM_MONITOR_NAME, metrics=metrics, tags=tags, schedule = schedule, monitor_runtime = monitor_runtime, background_mode=False).result
        else:
            custom_monitor_details = wos_client.monitor_definitions.add(name=CUSTOM_MONITOR_NAME, metrics=metrics, tags=tags, background_mode=False).result
    else:
        # otherwise, send the existing definition
        custom_monitor_details = existing_definition
    return custom_monitor_details

In [ ]:
custom_monitor_details = create_custom_monitor_definition()
print(custom_monitor_details)
custom_monitor_id = custom_monitor_details.metadata.id
custom_monitor_id


### Check the existence of custom monitor instance.


In [ ]:
def get_custom_monitor_instance(custom_monitor_id):
    monitor_instances = wos_client.monitor_instances.list(data_mart_id = WOS_GUID, monitor_definition_id = custom_monitor_id, target_target_id = subscription_id).result.monitor_instances
    if len(monitor_instances) == 1:
        return monitor_instances[0]
    return None

In [ ]:
# Openscale MRM service invokes custom metrics deployment url during runtime and wait for the default time of 60 second's to 
# to check the run status ie finished/Failed and fetch the latest measurement. Increase the wait time, if the runtime deployment 
# takes more than 60 seconds to compute and publish the custom metrics 

#Update the wait time here.
custom_metrics_wait_time = 300 #time in seconds <update the time here>

#Maximum number of records to consider during custom monitor run evaluation. Update max_records value as per the requirement
max_records = None
#Minimum number of records to consider during custom monitor run evaluation. Update min_records value as per the requirement
min_records = None

### Update the custom monitor instance.

In [ ]:
def update_custom_monitor_instance(custom_monitor_instance_id):
    payload = [
     {
       "op": "replace",
       "path": "/parameters",
       "value": {
           "custom_metrics_provider_id": integrated_system_id,
           "custom_metrics_wait_time":   custom_metrics_wait_time,
            "enable_custom_metric_runs": True
       }
     }
    ]
    if max_records is not None:
        payload[0]["value"]["max_records"] = max_records    
    if min_records is not None:
        payload[0]["value"]["min_records"] = min_records 
        
    response = wos_client.monitor_instances.update(custom_monitor_instance_id, payload, update_metadata_only = True)
    result = response.result
    return result

### For the custom monitor definition, create a custom monitor instance.


In [ ]:
def create_custom_monitor_instance(custom_monitor_id):
    # Check if an custom monitor instance already exists
    existing_monitor_instance = get_custom_monitor_instance(custom_monitor_id)

    # If it does not exist, then create one
    if existing_monitor_instance is None:
        target = Target(
                target_type=TargetTypes.SUBSCRIPTION,
                target_id=subscription_id
            )
        parameters = {
            "custom_metrics_provider_id": integrated_system_id,
            "custom_metrics_wait_time":   custom_metrics_wait_time,
            "enable_custom_metric_runs": True
        }
        if max_records is not None:
            parameters["max_records"] =   max_records
        if min_records  is not None:
            parameters["min_records"] =   min_records 
        
        #Update your custom monitor metric ids in the below thresholds to update the default value
        thresholds = [ MetricThresholdOverride(metric_id=CUSTOM_MONITOR_METRICS_NAMES[0], type = MetricThresholdTypes.LOWER_LIMIT, value=0.85),
                       MetricThresholdOverride(metric_id=CUSTOM_MONITOR_METRICS_NAMES[1], type = MetricThresholdTypes.LOWER_LIMIT, value=0.7),
                       MetricThresholdOverride(metric_id=CUSTOM_MONITOR_METRICS_NAMES[2], type = MetricThresholdTypes.UPPER_LIMIT, value=0.95)
                     ]
        # create the custom monitor instance id here.
        custom_monitor_instance_details = wos_client.monitor_instances.create(
                    data_mart_id=WOS_GUID,
                    background_mode=False,
                    monitor_definition_id=custom_monitor_id,
                    target=target,
                    parameters=parameters,
                    thresholds = thresholds
        ).result
    else:
        # otherwise, update the existing one with latest integrated system details.
        instance_id = existing_monitor_instance.metadata.id
        custom_monitor_instance_details = update_custom_monitor_instance(instance_id)
    return custom_monitor_instance_details

In [ ]:
monitor_instance_details = create_custom_monitor_instance(custom_monitor_id)
custom_monitor_instance_id = monitor_instance_details.metadata.id
print(monitor_instance_details)


### Run the custom monitor <a name="run"></a>


In [ ]:
#Execute the custom metrics provider deployment
monitor_instance_run_info = wos_client.monitor_instances.run(
        background_mode=False,
        monitor_instance_id=custom_monitor_instance_id
     ).result

monitor_instance_run_info
custom_monitor_run_id = monitor_instance_run_info.metadata.id


### Show Custom Metrics

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=custom_monitor_instance_id)

## Recap of the steps performed in this notebook

- Create a python function
- Deploy the python function to WML
- Create an OpenScale Integrated System pointing to the python function
- Create a Custom Monitor Definition mentioning various custom metrics
- Create a Custom Monitor Instance and specify the Integrated System ID in the monitor instance configuration.
- Run the Custom monitor

## Congratulations

You have finished configuring Custom Monitor Definition and Monitor instance and executing Custom Monitor Run for your Subscription. You can also run the custom monitor from `Watson OpenScale Dashboard`(https://url-to-your-cp4d-cluster/aiopenscale). Click the tile of your model and select `Evaluate Now` option from `Actions` drop down menu to run the custom monitor.